In [1]:
import itertools
from random import random, randint
import texttable
import operator


In [10]:
# the fixture function finds the result of the two teams and updates the team class as appropriate
def football_fixture(team1, team2, draws_allowed):
    # use rank to set win conditions
    delta = int(team1.rank) - int(team2.rank)
    # positive delta = team 1 better than team 2
    # negative delta = team 2 better than team 1
    if delta >= 0:
        win_condition_1 = 0.0006 * delta + 0.35
        win_condition_2 = 1 - (-0.0003 * delta + 0.35)
    else:
        win_condition_1 = -0.0003 * abs(delta) + 0.35
        win_condition_2 = 1 - (0.0006 * abs(delta) + 0.35)
        
    if not draws_allowed:
        draw_probability = win_condition_2 - win_condition_1
        win_condition_1 += draw_probability/2
        win_condition_2 -= draw_probability/2        

    r = random()

    if r < win_condition_1:  # team 1 won
        if draws_allowed:
            team1.points += 3
            team1.result = team1.result + 'W'
            team2.result = team2.result + 'L'
            
        if delta >= 0:
            # this is the most likely result
            team1_goals = randint(1, int(round(0.0027 * delta + 2)))
            team2_goals = min([team1_goals - 1, int(round(-0.0009 * delta + 2))])
        else:
            team1_goals = randint(1, int(round(-0.0009 * abs(delta) + 2)))
            team2_goals = min([team1_goals - 1, int(round(0.0027 * abs(delta) + 2))])

    elif r < win_condition_2 and draws_allowed:
        # then we have a draw
        team1.points += 1
        team1.result = team1.result + 'D'
        team2.points += 1
        team2.result = team2.result + 'D'
        draw_goals = randint(0, int(round(-0.0036 * delta + 3)))
        team1_goals = draw_goals
        team2_goals = draw_goals

    else:  # team 2 won the game
        if draws_allowed:
            team2.points += 3
            team2.result = team2.result + 'W'
            team1.result = team1.result + 'L'
            
        if delta >= 0:
            # this is the most likely result
            team2_goals = randint(1, int(round(0.0027 * delta + 2)))
            team1_goals = min([team2_goals - 1, int(round(-0.0009 * delta + 2))])
        else:
            team2_goals = randint(1, int(round(-0.0009 * abs(delta) + 2)))
            team1_goals = min([team2_goals - 1, int(round(0.0027 * abs(delta) + 2))])

    # update the goal differences for each team for group stages
    if draws_allowed:
        team1.goals_forward += team1_goals
        team1.goals_against += team2_goals
        team1.goal_difference = team1.goals_forward - team1.goals_against
        team2.goals_forward += team2_goals
        team2.goals_against += team1_goals
        team2.goal_difference = team2.goals_forward - team2.goals_against
    
    # update the fixture history
    team1.fixtures += team2.team + ', ' + team1.result[-1]
    team2.fixtures += team1.team + ', ' + team2.result[-1]
    # report result  
    print('                        ' + team1.code + '  ' + str(team1_goals) + ' - ' + str(team2_goals) + '  ' + team2.code)

In [3]:
# the group_stage function resolves all the fixtures within a group stage and stores the results inside the team class
def group_stages(teams_in_group):
    for i in range(8):  # 8 groups A to H        
        this_group = teams_in_group[i]
        print('\n\n                            GROUP ' + this_group[0].group) 
        # find all the fixtures for this group
        group_fixtures = list(itertools.combinations(this_group, 2))
        # play each fixture
        for j in range(len(group_fixtures)):
            team1, team2 = group_fixtures[j]
            football_fixture(team1, team2, draws_allowed=True)
        
        # determine group winners
        # points, GD, GF, points among those tied, GD among those tied.
        sorted_group = sorted(this_group, key=operator.attrgetter('points'))
        sorted_group.reverse()
                
        # build league tales using text table     
        headers = ['Team', 'Form', 'GF', 'GD', 'Pts']
        table = texttable.Texttable()
        table.header(headers)
        for x in range(4):
            table.add_row([sorted_group[x].code, sorted_group[x].result, sorted_group[x].goals_forward, sorted_group[x].goal_difference, sorted_group[x].points])
            
        table.set_cols_align(['l', 'c', 'c', 'c', 'c'])
        table.set_cols_width([8, 7, 7, 7, 7])
        table.set_chars(['-', '', '', '"'])
        print(table.draw())


In [ ]:

# the Finals function resolves the knock out stages of the finals
def Finals(group_winners)
    Fixture(T1, T2, draws_allowed=False)


SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)

In [4]:
class Team:
    def __init__(self, team, code, rank, group):
        self.team = team
        self.code = code
        self.rank = rank
        self.group = group
        self.fixtures = []
        self.points = 0
        self.result = ''
        self.goal_difference = 0
        self.goals_forward = 0
        self.goals_against = 0
                
    def __repr__(self):
        return f'{self.team}'


In [11]:

# input data
# team name, FIFA rank, group ID
# 'England', 8, 'E'

teams_list = []
# load csv
with open('teams.csv', 'r') as f:
    for line in f.readlines()[1:]:
        team, code, rank, group = line.strip().split(',')
        # define Team class
        teams_list.append(Team(team, code, rank, group))
        
# isolate the teams by the group they appear in
teams_in_group = [(list(g)) for _, g in itertools.groupby(teams_list, lambda x: x.group)]

# simulate the group stages
group_stages(teams_in_group)




                            GROUP A
                        RUS  1 - 0  SAU
                        RUS  0 - 1  EGY
                        RUS  1 - 0  URU
                        SAU  1 - 2  EGY
                        SAU  5 - 5  URU
                        EGY  0 - 1  URU
----------------------------------------------
   Team     Form      GF       GD       Pts   
""""""""""""""""""""""""""""""""""""""""""""""
 EGY         WWL       3        1        6    
----------------------------------------------
 RUS         WLW       2        1        6    
----------------------------------------------
 URU         LDW       6        0        4    
----------------------------------------------
 SAU         LLD       6       -2        1    
----------------------------------------------


                            GROUP B
                        POR  0 - 0  SPA
                        POR  4 - 1  MOR
                        POR  1 - 0  IRA
                        SPA  2 - 1  MOR
       

In [ ]:




winner = Finals(group_winners)


SyntaxError: invalid syntax (<ipython-input-45-caf27de49540>, line 3)